In [9]:
from skimage.io import collection, imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
import glob

from datetime import datetime
import nltk

# How is the public opinion about a company correlated to it's market value?

A company's market value is variable and depends on a lot of factors. The price is a reflection of the company's perceived value – what the public is willing to pay for a piece of the company. It can and will rise and fall, based on a variety of factors in the global landscape and within the company itself. One of which is becoming more influential than ever - the public's opinion on social media.

To analyze this correlation we'll look at two datasets. The first one contains over 3 million unique tweets with their information such as tweet id, author of the tweet, post date, the text body of the tweet, and the number of comments, likes, and retweets of tweets matched with the related company.

The second one will just have daily stock price records (from the Forbes2000) for us to make a reference with.

### 1. Data Acquisition 

So first let's read the tweets dataset into pandas and inspect a small sample from the two dataframes.

In [10]:
tweets = pd.read_csv('./top-companies-tweets/Tweet.csv')
tweets.sample(5)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
2416970,994380343800197120,JRook365,1525913723,Join @RobinhoodApp and we'll both get a stock ...,0,0,1
2329988,979065894658891776,IzabelaMelamed,1522262474,$AMZN on the daily – is this what they call an...,0,0,0
2542037,1018910217772437504,valuewalk,1531762101,$amzn wow https://bloomberg.com/news/articles...,0,1,1
2931941,1074643519405465600,ElliottForecast,1545049955,$AMZN 1 hour pre open view from 12/12 update. ...,0,1,1
3138111,1106567401582133250,KirkLubimov,1552661202,In a marketing exercise Elon Musk unveils the ...,0,0,0


In [11]:
tweets_company = pd.read_csv('./top-companies-tweets/Company_Tweet.csv')
tweets_company.sample(5)

,tweet_id,ticker_symbol
4146354,1135573448527372288,TSLA
2016140,951462622926426112,GOOGL
2602033,982319539693043712,AMZN
3846514,1043704038670393344,TSLA
2709334,1069608137026662401,AMZN


So next up let's read the datasets for each of the stocks, which we are monitoring. We will save them in a dictionary with the key, being the company's tick name and the value - it's stock prices over time dataset.

In [12]:
stocks_df = {}
for name in glob.glob('./stocks/*'):
    stocks_df[name.split('\\')[-1].split('.')[0]] = pd.read_csv(name)
stocks = pd.concat(stocks_df)
stocks.sample(10)

Date          Low         Open     Volume         High  \
AAPL 2934  22-07-1992     0.392857     0.404018  161974400     0.406250   
GOOG 1030  22-09-2008   213.699005   226.217087    8847639   226.217087   
AAPL 6394  13-04-2006     2.350357     2.369286  734678000     2.408571   
     6469  01-08-2006     2.355000     2.400714  711765600     2.426071   
     2782  13-12-1991     0.444196     0.444196   95121600     0.453125   
GOOG 1647  04-03-2011   298.979340   303.029175    6044572   303.352966   
AAPL 4255  10-10-1997     0.191964     0.191964  270401600     0.203125   
AMZN 5987  02-03-2021  3087.120117  3143.469971    2595800  3163.520020   
     3172  22-12-2009   132.649994   133.759995    8257500   135.990005   
AAPL 2462  10-09-1990     0.319196     0.330357   75980800     0.330357   

                 Close  Adjusted Close  
AAPL 2934     0.395089        0.325636  
GOOG 1030   214.266876      214.266876  
AAPL 6394     2.373929        2.041349  
     6469     2.399286        2.063153  
     2782     0.449777        0.369276  
GOOG 1647   299.188568      299.188568  
AAPL 4255     0.202567        0.174188  
AMZN 5987  3094.530029     3094.530029  
     3172   133.750000      133.750000  
AAPL 2462     0.319196        0.258771

### 2. Data Tidying and Cleaning

First let's combine the two tables from the twitter dataset, convert the dates to a datetime object and rename the column.

In [13]:
tweets = pd.merge(tweets, tweets_company, on = "tweet_id")

In [14]:
tweets["date"] = pd.to_datetime(tweets.post_date, unit='s')
tweets = tweets.drop(columns="post_date")
tweets.sample(5)


,tweet_id,writer,body,comment_num,retweet_num,like_num,ticker_symbol,date
726578,659134511486279685,KennyTheBigS,$AAPL so when does apple report?https://itunes...,0,0,0,AAPL,2015-10-27 22:28:08
736956,660420244784021504,Trading5DiasCom,$AAPL $GOOG $MSFT $AMZN $FB Tech's Fab 5 are n...,0,1,0,MSFT,2015-10-31 11:37:11
1286611,745410638655033345,_peripherals,Judge tosses 'Error 53' lawsuit against #Apple...,0,0,0,AAPL,2016-06-22 00:18:39
2884687,1009446429700608000,reurope_stock,"Reaffirmed: Tesla $TSLA ""Sell"" Rating Kept by ...",0,0,0,TSLA,2018-06-20 14:42:38
3200479,1047854211071967232,Marshmall0wpeep,"""They say that all good things must end"" $TSLA...",0,0,2,TSLA,2018-10-04 14:21:26


We will see what timeframe does our dataset cover, by getting the data of the earliest and latest tweets.

In [15]:
tweets.date.min(), tweets.date.max()

(Timestamp('2015-01-01 00:00:57'), Timestamp('2019-12-31 23:55:53'))

So it has data from 01.01.2015 to 31.12.2019, so basically from 2015 to the beginning of 2020. Knowing this we can filter out the stock prices to be only in this period of time. But first we have to covert the "Date" column to datetime.

In [16]:
def string_to_date(date_string):
    return datetime.strptime(date_string, "%d-%m-%Y")
stocks.Date = pd.to_datetime(stocks.Date.apply(string_to_date))

In [17]:
stocks = stocks[(stocks.Date >= '01-01-2015') & (stocks.Date < '01-01-2020')]
stocks.sample(10)

Date          Low         Open     Volume         High  \
GOOG 2911 2016-03-14   725.150024   726.809998    1718300   735.500000   
AAPL 8638 2015-03-16    30.717501    30.969999  143497200    31.237499   
GOOG 2714 2015-06-02   531.330017   532.929993    1939000   543.000000   
AMZN 5475 2019-02-19  1600.560059  1601.000000    3681700  1634.000000   
TSLA 1646 2017-01-11    45.335999    45.813999   18254000    45.995998   
     1171 2015-02-24    40.340000    41.458000   33018000    41.458000   
GOOG 2820 2015-10-30   710.049988   715.729980    1908800   718.000000   
     3078 2016-11-08   780.190002   783.400024    1366900   795.632996   
MSFT 8250 2018-12-03   110.730003   113.000000   34732800   113.419998   
TSLA 1500 2016-06-14    42.506001    43.776001   17901000    44.439999   

                 Close  Adjusted Close  
GOOG 2911   730.489990      730.489990  
AAPL 8638    31.237499       28.477116  
GOOG 2714   539.179993      539.179993  
AMZN 5475  1627.579956     1627.579956  
TSLA 1646    45.945999       45.945999  
     1171    40.821999       40.821999  
GOOG 2820   710.809998      710.809998  
     3078   790.510010      790.510010  
MSFT 8250   112.089996      108.796532  
TSLA 1500    42.992001       42.992001

As we can see we don't have data for every day, because the stock market funcitons only on workdays, unlike twitter.  We will find a way to work around this later.

Now lets strip the data down to just one column - the value, which we will calculate by getting the mean of the Open and Close prices. 

In [18]:
stocks["Value"] = (stocks.Open + stocks.Close) / 2
stocks = stocks.drop(columns=['Low', 'Open', 'High', 'Close', 'Adjusted Close'])
stocks.sample(10)

Date     Volume        Value
GOOG 3447 2018-04-30    1671300  1023.670013
MSFT 7550 2016-02-24   33014500    51.025000
AMZN 4535 2015-05-26    2244200   425.835007
TSLA 2013 2018-06-27   41668500    68.950001
GOOG 2630 2015-01-30    5606350   523.752045
AMZN 4819 2016-07-11    3195300   751.890015
AAPL 8926 2016-05-05  143562000    23.405000
     9596 2019-01-03  365248800    35.771250
MSFT 7749 2016-12-06   19907000    60.190001
GOOG 2859 2015-12-28    1515300   757.714996

It is a little inconvenient to have the stock name as an index instead of it being a regular column. We will fix that and also change the column names to match the twitter dataset.

In [19]:
stocks = stocks.reset_index(level=0)

In [20]:
stocks.columns = ["ticker_symbol", "date", "volume", "value"]
stocks.sample(10)
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6290 entries, 8589 to 2393
Data columns (total 4 columns):
ticker_symbol    6290 non-null object
date             6290 non-null datetime64[ns]
volume           6290 non-null int64
value            6290 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 245.7+ KB


 So next up lets take a look at the datatypes and null values for the twitter dataset.

In [21]:
tweets.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
tweet_id         4336445 non-null int64
writer           4280526 non-null object
body             4336445 non-null object
comment_num      4336445 non-null int64
retweet_num      4336445 non-null int64
like_num         4336445 non-null int64
ticker_symbol    4336445 non-null object
date             4336445 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 297.8+ MB


Everything looks good, except the ticker_symbol which should be a category. Also the writer column has quite a few missing records, but we won't be using it for our model and analysis, so we can discard it altogether.

In [26]:
tweets.ticker_symbol = tweets.ticker_symbol.astype('category')
tweets = tweets.drop(columns=["writer"])
tweets.head(7)

,tweet_id,body,comment_num,retweet_num,like_num,ticker_symbol,date
0,550441509175443456,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,2015-01-01 00:00:57
1,550441672312512512,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,2015-01-01 00:01:36
2,550441732014223360,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN,2015-01-01 00:01:50
3,550442977802207232,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA,2015-01-01 00:06:47
4,550443807834402816,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05
6,550443808606126081,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05
8,550443809700851716,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05


As we can see there seem to be a lot of duplicate bodies in our dataset. We want to remove them and this is exactly what the following code does. 

In [23]:
tweets = tweets.drop_duplicates(subset=["body"])

### 3. Text Preparation and Exploration

Before we start working with the text, we have to prepare it and take a quick look at some statistics about it.

False    3326194
Name: body, dtype: int64